# Query the Dataset in S3 using Amazon Athena and AWS Glue Catalog
This will create an Athena table in the Glue Catalog (Hive Metastore) which allows us to query the dataset files in S3.  We will create a table in the Glue Catalog based on the `Amazon Customer Reviews Dataset` in S3.  We define the columns that map to the data, specify how the data is formatted, and provide the location in Amazon S3 for the file(s).


<img src="https://github.com/data-science-on-aws/data-science-on-aws/blob/quickstart/00_quickstart/img/athena_register_parquet.png?raw=true" width="60%" align="left">

In [2]:
import boto3
import sagemaker

sess = sagemaker.Session()
# bucket = sess.default_bucket()
bucket = 'data-science-on-aws-applied'
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

In [3]:
print('Bucket:',bucket)
print('Region:',region)
print("Account:",account_id)

Bucket: data-science-on-aws-applied
Region: us-east-1
Account: 707031497630


In [4]:
from pyathena import connect
import pandas as pd

In [5]:
%store -r setup_dependencies_passed

In [6]:
try:
    setup_dependencies_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not install the required libraries.   ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [7]:
print(setup_dependencies_passed)

True


In [8]:
if not setup_dependencies_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not install the required libraries.   ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


In [9]:
%store -r role

In [10]:
role

'arn:aws:iam::707031497630:role/service-role/AmazonSageMaker-ExecutionRole-20220218T020780'

# Create Athena Table for Parquet Data

#### Dataset columns

- `marketplace`: 2-letter country code (in this case all "US").
- `customer_id`: Random identifier that can be used to aggregate reviews written by a single author.
- `review_id`: A unique ID for the review.
- `product_id`: The Amazon Standard Identification Number (ASIN).  `http://www.amazon.com/dp/<ASIN>` links to the product's detail page.
- `product_parent`: The parent of that ASIN.  Multiple ASINs (color or format variations of the same product) can roll up into a single parent.
- `product_title`: Title description of the product.
- `product_category`: Broad product category that can be used to group reviews (in this case digital videos).
- `star_rating`: The review's rating (1 to 5 stars).
- `helpful_votes`: Number of helpful votes for the review.
- `total_votes`: Number of total votes the review received.
- `vine`: Was the review written as part of the [Vine](https://www.amazon.com/gp/vine/help) program?
- `verified_purchase`: Was the review from a verified purchase?
- `review_headline`: The title of the review itself.
- `review_body`: The text of the review.
- `review_date`: The date the review was written.



In [11]:
s3_staging_dir = "s3://{}/athena/query-cache".format(bucket)

In [12]:
s3_staging_dir

's3://data-science-on-aws-applied/athena/query-cache'

In [41]:
database_name = "default"
table_name = "amazon_reviews_parquet"

In [42]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [19]:
conn

The statement only creates the table if it doesn't exist. For some reason, you can't just remove the `IF NOT EXISTS` portion. You have to go into Glue and delete the table, then recreate it. You can also do this from Athena.

In [43]:
# SQL statement to execute
statement = """
    CREATE EXTERNAL TABLE IF NOT EXISTS {}.{} (
      marketplace string, 
      customer_id string, 
      review_id string, 
      product_id string, 
      product_parent string, 
      product_title string, 
      star_rating int, 
      helpful_votes int, 
      total_votes int, 
      vine string, 
      verified_purchase string, 
      review_headline string, 
      review_body string, 
      review_date bigint, 
      year int)
    PARTITIONED BY (product_category string)
    ROW FORMAT SERDE 
      'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe' 
    STORED AS INPUTFORMAT 
      'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat' 
    OUTPUTFORMAT 
      'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat'
    LOCATION
      's3://data-science-on-aws-applied/customer-reviews/parquet/'
""".format(
    database_name, table_name
)

print(statement)

pd.read_sql(statement, conn)


    CREATE EXTERNAL TABLE IF NOT EXISTS default.amazon_reviews_parquet (
      marketplace string, 
      customer_id string, 
      review_id string, 
      product_id string, 
      product_parent string, 
      product_title string, 
      star_rating int, 
      helpful_votes int, 
      total_votes int, 
      vine string, 
      verified_purchase string, 
      review_headline string, 
      review_body string, 
      review_date bigint, 
      year int)
    PARTITIONED BY (product_category string)
    ROW FORMAT SERDE 
      'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe' 
    STORED AS INPUTFORMAT 
      'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat' 
    OUTPUTFORMAT 
      'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat'
    LOCATION
      's3://data-science-on-aws-applied/customer-reviews/parquet/'



""


# Analyze and Build the Partitions

In [44]:
statement = """
    MSCK REPAIR TABLE {}.{}
""".format(
    database_name, table_name
)

print(statement)

pd.read_sql(statement, conn)


    MSCK REPAIR TABLE default.amazon_reviews_parquet



""


# Verify The Table Has Been Created Succesfully

In [45]:
statement = "SHOW TABLES in {}".format(database_name)

df_tables = pd.read_sql(statement, conn)
df_tables

,tab_name
0,amazon_reviews_parquet
1,parquet_files


# Run A Sample Query

In [46]:
product_category = "Digital_Software"

statement = """
    SELECT * FROM {}.{}
    WHERE product_category = '{}' LIMIT 100
""".format(
    database_name, table_name, product_category
)

print(statement)


    SELECT * FROM default.amazon_reviews_parquet
    WHERE product_category = 'Digital_Software' LIMIT 100



In [47]:
df = pd.read_sql(statement, conn)
df.head(5)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,year,product_category
0,US,28476682,R1NABAMVOW15HT,B005HYI8R6,680721066,Quicken Deluxe 2012,5,0,3,N,N,Really good program,I really like this software it keep my finance...,15520,2012,Digital_Software
1,US,9971842,R21D1D59RLPVCJ,B008SCNF1S,531153669,Norton Antivirus 2013 - 1 User / 1 PC,1,0,0,N,Y,Worst product ever for Toshiba Satellite A105!...,"I'm going to put it plain and simple,<br />It ...",15898,2013,Digital_Software
2,US,14016292,R2I26R9I4A2WPN,B003LJXELE,527297648,Typing Instructor For Kids Platinum 5,1,1,2,N,Y,Doesn't install on Windows 7!!,I should have paid attention when other review...,15521,2012,Digital_Software
3,US,25482264,R2Z55MC3O13A9P,B00A42LPSK,835035048,H&R Block At Home Deluxe 2012,5,0,0,N,Y,Fast to download and easy to use,The item was fast and easy to download. All th...,15898,2013,Digital_Software
4,US,20368890,R26IG18C14MVR1,B007C89B44,78829106,Norton 360 6.0 - 1 User/3PC,5,0,0,N,Y,Very Good,Norton 360 increased my system speed and also ...,15522,2012,Digital_Software


In [48]:
if df.empty:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOUR DATA HAS NOT BEEN REGISTERED WITH ATHENA.")
    print("LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.             ")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


In [49]:
if "amazon_reviews_parquet" in df_tables.values and not df.empty:
    ingest_create_athena_table_parquet_passed = True

In [50]:
%store ingest_create_athena_table_parquet_passed

Stored 'ingest_create_athena_table_parquet_passed' (bool)


# Review the New Athena Table in the Glue Catalog

In [51]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="top" href="https://console.aws.amazon.com/glue/home?region={}#table:catalog={};name=amazon_reviews_parquet;namespace=default">AWS Glue Catalog</a></b>'.format(
            region, account_id
        )
    )
)

/tmp/ipykernel_1483/2118095021.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Store Variables for the Next Notebooks

In [52]:
%store

Stored variables and their in-db values:
ingest_create_athena_table_parquet_passed             -> True
role                                                  -> 'arn:aws:iam::707031497630:role/service-role/Amazo
setup_dependencies_passed                             -> True


# Release Resources

In [53]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>